#Before Starting

For the first point, We started downloading the yellow cab data for January 2018(yellow_tripdata_2018-01.csv). Our plan was to make our studies and analysis firstly for one month and then extend our results considering also other years.
Our Data-set contains a really large amount of data so we opted not to choose the simple pd.read_csv() module from pandas. After considering many options such as pandas with the chuncksize option, we decided to use the Dask DataFrames. This different class has some some similarities with the usual Pandas dataframes but has some pros dealing with memory and speed. 
So the code we used is: 

In [1]:
import dask.dataframe as dd
df = dd.read_csv('yellow_tripdata_2018-01.csv')

After checking that the import operation ended correctly, we had to merge our datasets with a dataset contained in an HTML page(https://github.com/CriMenghini/ADM-2018/blob/master/Homework_2/taxi_zone_lookup.csv).
The data we needed was not the only thing in the page so we searched for a method to be able to extract all the 256 rows with 4 attributes web-scraping the sourcecode. Using the Beautifoul Soup library and with a bit of googling, we found out the correct sintax to find and store all the variables in a new dataset. 

((((THIS IS FOR US: I DO REALIZE THAT MY CODE PROBABLY IS NOT EFFICIENT BUT IT IS THE ONLY THING I WAS ABLE TO COME UP WITH)))

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
page = requests.get("https://github.com/CriMenghini/ADM-2018/blob/master/Homework_2/taxi_zone_lookup.csv")
soup = BeautifulSoup(page.content, 'html.parser')
ids=[]
bor=[]
zon=[]
srv_zon=[]
cell=0
for i in range(2,1327,5):    #FIRSTLY We get all the location ids
    a=soup.find_all('td')[i].get_text()  
    ids.append(a)
    cell=cell+1
    
cell=0
for i in range(3,1328,5):    #Then all the boroughs
    a=soup.find_all('td')[i].get_text()
    bor.append(a)
    cell=cell+1
    
cell=0
for i in range(4,1329,5):   #After that we get al zones
    a=soup.find_all('td')[i].get_text()
    zon.append(a)
    cell=cell+1
cell=0
for i in range(5,1330,5):    #Finally we get all the seving zones
    a=soup.find_all('td')[i].get_text()
    srv_zon.append(a)
    cell=cell+1
# after getting all the informations needed we merged all the lists together
data_tuples = list(zip(ids,bor,zon,srv_zon)) 
second=pd.DataFrame(data_tuples, columns=['PULocationID','Borough','Zone','service_zone'])

We then proceeded merging the two dataframes obtaines(second and df). BUbt first, we converted the Dask DataFrame back to a Pandas One. STILL A LOT OF THINGS CAN BE WRITTEN HERE. I changed all the NaN to 0 because in my opinion they can still be useful for our analysis.  

In [4]:
df1 = df.compute()  #Dask datFrame back to pandas
frames = [df1,second]
result = pd.concat(frames,sort=True)  #Merging
result1=result.fillna(value=0)
print(result1.head())

  Borough  DOLocationID PULocationID  RatecodeID  VendorID Zone  extra  \
0       0          24.0           41         1.0       1.0    0    0.5   
1       0         140.0          239         1.0       1.0    0    0.5   
2       0         141.0          262         1.0       1.0    0    0.5   
3       0         257.0          140         1.0       1.0    0    0.5   
4       0         239.0          246         1.0       1.0    0    0.5   

   fare_amount  improvement_surcharge  mta_tax  passenger_count  payment_type  \
0          4.5                    0.3      0.5              1.0           2.0   
1         14.0                    0.3      0.5              1.0           2.0   
2          6.0                    0.3      0.5              2.0           1.0   
3         33.5                    0.3      0.5              1.0           2.0   
4         12.5                    0.3      0.5              2.0           1.0   

  service_zone store_and_fwd_flag  tip_amount  tolls_amount  total_a